In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
import time
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [13]:
# dataset class for correct dataloader initialization
class TumobrainorDataset(Dataset):
    def __init__(self, images, labels) -> None:
        self.X = images
        self.y = labels

        self.random_transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(256),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.RandomRotation(degrees=45),
            transforms.ColorJitter(
                brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        num_augment = 8
        augmented_batch = []
        for i in range(num_augment):
            new_item = self.random_transform(self.X[index])
            augmented_batch.append(new_item)

        labels = torch.zeros(4, dtype=torch.float32)
        labels[int(self.y[index]) - 1] = 1

        new_labels = [labels, labels, labels,
                      labels, labels, labels, labels, labels]

        return torch.stack(augmented_batch), torch.stack(new_labels)

In [14]:
# loading all dataloaders
with open("./dataset/train_loader.pickle", "rb") as f:
    train_loader = pickle.load(f)
with open("./dataset/valid_loader.pickle", "rb") as f:
    valid_loader = pickle.load(f)
with open("./dataset/test_loader.pickle", "rb") as f:
    test_loader = pickle.load(f)

## Building model

In [15]:
# loading pretrained ResNet50 model
weights = torchvision.models.ResNet50_Weights.DEFAULT
resnet = torchvision.models.resnet50(weights=weights)

In [16]:
# unfreezing model's parameters
for param in resnet.parameters():
    param.requires_grad = True

n_inputs = resnet.fc.in_features
# modifying last layer of model
resnet.fc = nn.Sequential(nn.Linear(n_inputs, 2048),
                          nn.SELU(),
                          nn.Dropout(p=.4),
                          nn.Linear(2048, 2048),
                          nn.SELU(),
                          nn.Dropout(p=0.4),
                          nn.Linear(2048, 4),
                          nn.LogSigmoid())

for name, child in resnet.named_children():
    for name2, params in child.named_parameters():
        params.requires_grad = True

In [17]:
# use GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [18]:
resnet.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## Model training

In [19]:
# defining loss function and optimizer
lossfunc = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(resnet.parameters(), lr=3e-4)

train_losses, test_losses, train_correct, test_correct = [], [], [], []

In [20]:
def save_checkpoint(state, is_best, filename="./best models/resnet_checkpoint.pth.tar"):
    if is_best:
        torch.save(state, filename)

In [21]:
epochs = 15

start_time = time.time()

best_prec1 = 2

batch_num, train_b, test_b = None, None, None

for epoch_i in range(epochs):
    train_corr, test_corr = 0, 0

    epoch_start = time.time()
    
    for batch_num, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)

        y_pred = resnet(X.view(-1, 3, 224, 224))
        loss = lossfunc(y_pred.float(), torch.argmax(y.view(32, 4), dim=1).long()) # 32 samples in batch, 8 images in one sample
        # find predicted labels
        predicted = torch.argmax(y_pred, dim=1).data

        # calculate the amount of correct predictions in batch
        batch_corr = (predicted == torch.argmax(y.view(32, 4), dim=1)).sum()
        # add batch corrects to the total amount in training epoch
        train_corr += batch_corr
        
        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    epoch_end = time.time() - epoch_start
    print(f"Epoch {epoch_i + 1} | Batch {(batch_num + 1) * 4}\nAccuracy: {train_corr.item() * 100 / (4 * 8 * batch_num):2.2f} | Loss: {loss.item():2.4f} | Duration: {epoch_end / 60:.2f} minutes")

    train_b = batch_num
    train_losses.append(loss)
    train_correct.append(train_corr)

    X, y = None, None

    with torch.no_grad():
        for batch_num, (X, y) in enumerate(valid_loader):
            X, y = X.to(device), y.to(device)

            y_val = resnet(X.view(-1, 3, 224, 224))
            predicted = torch.argmax(y_val, dim=1).data

            test_corr += (predicted == torch.argmax(y.view(32, 4), dim=1)).sum()
    
    loss = lossfunc(y_val.float(), torch.argmax(y.view(32, 4), dim=1).long())

    print(f"Validation Accuracy: {test_corr.item() * 100 / (4 * 8 * batch_num):2.2f} | Validation loss: {loss.item():2.4f}\n")

    is_best = loss < best_prec1
    best_prec1 = min(loss, best_prec1)
    save_checkpoint({
        "epoch": epoch_i + 1,
        "state_dict": resnet.state_dict(),
        "best_prec1": best_prec1,
    }, is_best)

    test_b = batch_num
    test_losses.append(loss)
    test_correct.append(test_corr)

end_time = time.time() - start_time

print(f"\nTraining Duration: {end_time / 60:.2f} minutes")
print(f"GPU memory used: {torch.cuda.memory_allocated()} kb")
print(f"GPU memory cached: {torch.cuda.memory_cached()} kb")

Epoch 1 | Batch 2828
Accuracy: 84.87 | Loss: 0.2272 | Duration: 26.40 minutes
Validation Accuracy: 90.69 | Validation loss: 0.2520

Epoch 2 | Batch 2828
Accuracy: 93.09 | Loss: 0.0162 | Duration: 26.26 minutes
Validation Accuracy: 95.38 | Validation loss: 0.0563

Epoch 3 | Batch 2828
Accuracy: 94.30 | Loss: 0.0726 | Duration: 25.93 minutes
Validation Accuracy: 95.17 | Validation loss: 1.0297

Epoch 4 | Batch 2828
Accuracy: 95.83 | Loss: 0.0007 | Duration: 25.81 minutes
Validation Accuracy: 96.10 | Validation loss: 0.6742

Epoch 5 | Batch 2828
Accuracy: 96.49 | Loss: 0.1081 | Duration: 25.89 minutes
Validation Accuracy: 93.75 | Validation loss: 0.0401

Epoch 6 | Batch 2828
Accuracy: 97.05 | Loss: 0.0461 | Duration: 25.74 minutes
Validation Accuracy: 97.94 | Validation loss: 0.0009

Epoch 7 | Batch 2828
Accuracy: 97.26 | Loss: 0.0026 | Duration: 25.86 minutes
Validation Accuracy: 95.46 | Validation loss: 1.2154

Epoch 8 | Batch 2828
Accuracy: 97.49 | Loss: 0.0001 | Duration: 25.86 minute

c:\Users\husai\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\cuda\memory.py:416: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


## Model testing

In [22]:
# saving last version of the model
torch.save(resnet.state_dict(), './best models/last_model.pt')

In [47]:
# initializing new model
weights = torchvision.models.ResNet50_Weights.DEFAULT
best_model = torchvision.models.resnet50(weights=weights)

for param in best_model.parameters():
    param.requires_grad = True

n_inputs = best_model.fc.in_features

best_model.fc = nn.Sequential(nn.Linear(n_inputs, 2048),
                          nn.SELU(),
                          nn.Dropout(p=.4),
                          nn.Linear(2048, 2048),
                          nn.SELU(),
                          nn.Dropout(p=0.4),
                          nn.Linear(2048, 4),
                          nn.LogSigmoid())


for name, child in best_model.named_children():
    for name2, params in child.named_parameters():
        params.requires_grad = True

In [49]:
# loading params of best model
loaded_model = torch.load("./best models/resnet_checkpoint.pth.tar")
best_model.load_state_dict(loaded_model["state_dict"])
best_model.eval()
best_model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [50]:
# test model with no gradient updates
with torch.no_grad():
    correct = 0
    test_loss = []
    test_corr = []
    labels = []
    pred = []
    # perform test set evaluation batch wise
    for (X, y) in test_loader:
        # set label to use CUDA if available
        X, y = X.to(device), y.to(device)

        # append original labels
        labels.append(torch.argmax(y.view(32, 4), dim=1).data)

        # perform forward pass
        y_val = best_model(X.view(-1, 3, 224, 224))

        # get argmax of predicted values, which is our label
        predicted = torch.argmax(y_val, dim=1).data
        # append predicted label
        pred.append(predicted)

        # calculate loss
        loss = lossfunc(y_val.float(), torch.argmax(y.view(32, 4), dim=1).long())

        # increment correct with correcly predicted labels per batch
        correct += (predicted == torch.argmax(y.view(32, 4), dim=1)).sum()

        # append correct samples labels and losses
        test_corr.append(correct)
        test_loss.append(loss)
        
print(f"Test Loss: {test_loss[-1].item():.4f}")
print(f"Test accuracy: {test_corr[-1].item() * 100 / (4 * 8 * 151)}")

Test Loss: 0.0010
Test accuracy: 98.30298013245033


In [51]:
# saving the best model without unnecessary data
torch.save(best_model.state_dict(), './best models/best_tumobrainor.pt')